In [1]:
import requests
import re
from lxml import etree
import time
import csv
from bs4 import BeautifulSoup
import json
import pickle

In [2]:
headers = {
"cookie":"BAIDU_SSP_lcr=https://cn.bing.com/; SUB=_2A25OxGW4DeRhGeBG7FsQ9SnKzzuIHXVqRwvwrDV6PUJbktANLUKhkW1NQefZ6IjAuIu5ei0HfjjD0nq8JHq62f5u; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9WWcUF0gHI_M8h5ywJreSgBx5NHD95Qc1hM4eK-NSoBNWs4DqcjEi--Xi-i2i-27i--Xi-zRiKy2i--Ri-z7iK.4i--NiKLWiKnXi--RiKyFi-2c1KBEeK.t; SSOLoginState=1673532904; _T_WM=55736388746; WEIBOCN_FROM=1110006030; MLOGIN=1; BAIDU_SSP_lcr=https://cn.bing.com/; __bid_n=185a8cf0148ab105024207; FPTOKEN=DFj0i4fBOm0iJp11KG1ftoOTQ7Hn88cjf3GkjuYd3hC5q+zJXAk+JWdL+jStk3FzvQ5lj+crzXrcTFC/EitxHMbGQpV/ul+BcpPtfkzhjoSSdBluN5l6cIDgL/kstqh+tQFnVgRj3cK4YEaYupHI6Kv5pLvuynkrRsUswQHwi43D07XHmDUodfrG+H3oDV0o1EDgDYpkmz4PAakm3+AWeuXZlkGtGsckl5/KHDvdhGzqttI+BKBauFax4XA/JR9p/cyJmXrw0uNbwJZZZi/NKBmzJKOQo4/giGLWcDEn69ZdLIrZm9+znSZz1uuSSwBopF5uYFly3kAkV/avB0k7hDk+ETKVNbZ+4oF/f05Y8lpV+fXDkJyCNoNM02f7afm07lsJefiBYItVdspVZ+28qQ==|QoQ/qKJIsbFv9vEcKhrMmc1BQ80s+A5E/q22MX8ChEw=|10|ecaa752b980fbae8d1349f457888ddae; XSRF-TOKEN=adde7f; M_WEIBOCN_PARAMS=oid%3D4857105729459510%26luicode%3D20000061%26lfid%3D4857105729459510",
"referer": "https://m.weibo.cn/",
"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.76"
}
text_url = 'https://m.weibo.cn/detail/4857105729459510'
# content_url = 'https://m.weibo.cn/comments/hotflow?id=4857105729459510&mid=4857105729459510&max_id=141476389057508&max_id_type=0'
content_url = 'https://m.weibo.cn/comments/hotflow?id=4857105729459510&mid=4857105729459510&max_id_type=0'
# 下一条评论的url
next_content_url = ''

#数据结构
output_data = {
    'event': None, 
    'post': 'xxx',
    'time': 'xxxx-xx-xx', 
    'comments': []
}
one_comment = {
    'content': 'xxx', # 评论内容
    'time': 'xxxx-xx-xx', # 评论时间
    'ip': 'xxx' # 评论发布者的IP位置信息
}

In [3]:
def get_month(m_str):
    mouth = ''
    # print(m_str)
    if m_str =='Jan':
        mouth='01'
    elif m_str =='Feb':
        mouth='02'
    elif m_str =='Mar':
        mouth='03'
    elif m_str =='Apr':
        mouth='04'
    elif m_str =='May':
        mouth='05'
    elif m_str =='Jun':
        mouth='06'
    elif m_str =='Jul':
        mouth='07'
    elif m_str =='Aug':
        mouth='08'
    elif m_str =='Sept':
        mouth='09'
    elif m_str =='Oct':
        mouth='10'
    elif m_str =='Nov':
        mouth='11'
    elif m_str =='Dec':
        mouth='12'
    return mouth

def get_next_comtent_url(get_comtent_num,content_url,max_id,max_id_type):
    next_comtent_url = ''
    if(get_comtent_num == 0):
        content_url = content_url[:-14]
    else:
        content_url_list = content_url.split('&')
        content_url = content_url_list[0] +'&' + content_url_list[1]
    # content_url_list = content_url.split('&')
    # content_url = content_url_list[0] +'&' + content_url_list[1]
    next_comtent_url = content_url + f"&max_id={max_id}" + max_id_type
    print(next_comtent_url)
    # print(max_id)
    return next_comtent_url


In [4]:
def get_info(text_url,content_url,max_epoch,filename):
    #获取文章信息
    text_resp = requests.get(text_url,headers=headers)
    tmp_text_resp = text_resp.text
    # print(len(text_resp))
    soup = BeautifulSoup(tmp_text_resp,'lxml') 
    # print(soup)
    tag_list = soup.find_all('script')
    # print(len(tag_list))
    id = 0
    for tag in tag_list:
        # print(tag.text)
        post = re.findall(r'.*"text": "(.*)"',tag.text)
        # post = post[0]
        if(post):
            # print(type(post))
            post = post[0]
            # print(post)
            post = re.sub("<a.*", "", post)

            event = post.split("#")
            if(len(event) == 1):
                event="None"
            else:
                event = event[0]

            text_time = re.findall(r'.*"edit_at": "(.*)"',tag.text)
            text_time = text_time[0].split()
            # print(time)
            text_time = text_time[-1] + '-' + get_month(text_time[-5]) + '-' + text_time[-4]
            break
    # print(event)
    # print(post)
    # print(text_time)
    output_data['event'] = event
    output_data['post'] = post
    output_data['time'] = text_time
    #获取评论信息
    for comtent_num in range(max_epoch):
        print("当前执行至epoch:", comtent_num)
        content_resp = requests.get(content_url,headers=headers).json()
        tmp_data = content_resp['data']
        # 获取下一组评论的url
        max_id = str(tmp_data['max_id'])
        max_id_type = "&max_id_type="+str(tmp_data['max_id_type'])
        content_url = get_next_comtent_url(comtent_num,content_url,max_id,max_id_type)
        # 解析评论数据
        data = tmp_data['data']
        for item in data:
            content = item['text']
            # result = re.sub(r'<span.*</span>?','',content)
            # if result:
            #     content = result
            # print(content)
            content = re.sub("[^\u4e00-\u9fa5\u3002*\uff0c*\uff1f*]", "", content)

            tmp_time = item['created_at'].split()
            the_time = tmp_time[-1] + '-' + get_month(tmp_time[-5]) + '-' + tmp_time[-4]
            ip = re.sub(r'来自','',item['source'])
            like = item['like_count']
            one_comment = {
            'content': content,
            'time': the_time,
            'ip': ip,
            'like':like
            }
            output_data["comments"].append(one_comment)
        time.sleep(1)

    #将评论和文章信息写入文件
    with open(filename, 'w') as f:
        json.dump(output_data, f)
        f.close()

In [5]:
max_epoch = 10
filename = 'weibo.json'
get_info(text_url,content_url,max_epoch,filename)

当前执行至epoch: 0
https://m.weibo.cn/comments/hotflow?id=4857105729459510&mid=4857105729459510&max_id=141476389057508&max_id_type=0
当前执行至epoch: 1
https://m.weibo.cn/comments/hotflow?id=4857105729459510&mid=4857105729459510&max_id=139277365389399&max_id_type=0
当前执行至epoch: 2
https://m.weibo.cn/comments/hotflow?id=4857105729459510&mid=4857105729459510&max_id=138865049263323&max_id_type=0
当前执行至epoch: 3
https://m.weibo.cn/comments/hotflow?id=4857105729459510&mid=4857105729459510&max_id=138590170998819&max_id_type=0
当前执行至epoch: 4
https://m.weibo.cn/comments/hotflow?id=4857105729459510&mid=4857105729459510&max_id=138177858233348&max_id_type=0
当前执行至epoch: 5
https://m.weibo.cn/comments/hotflow?id=4857105729459510&mid=4857105729459510&max_id=138177856441485&max_id_type=0
当前执行至epoch: 6
https://m.weibo.cn/comments/hotflow?id=4857105729459510&mid=4857105729459510&max_id=138177856323439&max_id_type=0
当前执行至epoch: 7
https://m.weibo.cn/comments/hotflow?id=4857105729459510&mid=4857105729459510&max_id=138177

In [6]:
with open(filename, "rb") as fo:
   A = json.load(fo, encoding='bytes')
   print(A)

{'event': 'None', 'post': '【王思聪疑因打人被拘提请行政复议 律师：复议机关受理后应在60日内作出决定】1月12日，上海静安警方发布通报，称11日4时许接报南京西路一商务楼门口有人被打。因王某某等提请行政复议，警方对四人暂缓执行行政拘留。据新京报我们视频报道，有权威人士证实事件中的王某某正是王思聪。河南泽槿律师事务所主任付建律师表示，行政复议被受理后，复议机关应当自受理之日起60日内作出行政复议决定，情况复杂的，经复议机关负责人批准，可以适当延长，并告知申请人和被申请人，但是延长期限最多不超过30日，“如果申请人不服行政复议决定的，自收到行政复议决定书之日起15日内，可向人民法院提起行政诉讼。”', 'time': '2023-01-12', 'comments': [{'content': '七天又不影响过年，复议啥啊', 'time': '2023-01-12', 'ip': '江苏', 'like': 1929}, {'content': '果然公子哥就是不一样', 'time': '2023-01-12', 'ip': '北京', 'like': 1380}, {'content': '看看法大还是钱大哈哈赞', 'time': '2023-01-12', 'ip': '浙江', 'like': 1125}, {'content': '换做别人哪会有机会提出行政复议，不都是直接拉进拘留所哈哈哈哈', 'time': '2023-01-12', 'ip': '广东', 'like': 731}, {'content': '钞能力一下就体现出来了', 'time': '2023-01-12', 'ip': '四川', 'like': 489}, {'content': '不明白为啥评论戾气那么重啊，对于行政处罚提起行政复议是每个公民的权利，不是因为他有钱他才能提起来，换谁挨处罚了都能。乱停车那个罚单上不都明确写着呢可以提出复议啊', 'time': '2023-01-12', 'ip': '河南', 'like': 351}, {'content': '钱是个好东西', 'time': '2023-01-12', 'ip': '北京', 'like': 232}, {'content': '老王在里面消停天也不是什么坏事', 'time